In [1]:
# !pip install contractions
!pip install sklearn
!pip install pandas
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:
from data_utlis import *

In [4]:
input_path = '../data/'
df = pd.read_csv(input_path+'task_output_5313.txt', sep=',')
df.head(1)

,user_no,key_label,event,event_time,post_id,news_time,title,abstract,content,industry_name_sc,concept_name_sc
0,929,0,newsView,2022-07-28 23:56:17,13011492,2022-07-28 23:28:38,Market Movers | Solar stocks surge Thursday af...,Solar stocks were surging on Thursday morning ...,"<div class=""ftEditor""><p>11:02 AM EDT, 07/28/2...","[""太阳能"",""电气设备及零件""]","[""法国巴黎银行持仓"",""昨日强势股"",""光伏太阳能"",""氢能源"",""荷兰全球保险持仓"",""..."


In [5]:
df.fillna(' ', inplace=True)
df.columns

Index(['user_no', 'key_label', 'event', 'event_time', 'post_id', 'news_time',
       'title', 'abstract', 'content', 'industry_name_sc', 'concept_name_sc'],
      dtype='object')

In [6]:
# remove puncautions 
df['industry'] = df['industry_name_sc'].apply(remove_punctaution)
df['concept'] = df['concept_name_sc'].apply(remove_punctaution)

In [7]:
# combine industru y and concept as tags
df['tags'] = df['industry'] + ',' + df['concept']
df['tags'] = df['tags'].apply(lambda x: x.replace(',', ' '))
df['tags'].head(2)

0    太阳能 电气设备及零件 法国巴黎银行持仓 昨日强势股 光伏太阳能 氢能源 荷兰全球保险持仓 ...
1    航运及港口 航运港口 一带一路 港口运输股 摘帽 转融券标的 国企改革 BDI指数 证金持股...
Name: tags, dtype: object

In [8]:
# remove puncaution
df['clean_content'] = df['content'].apply(remove_punc)
df['clean_title'] = df['title'].apply(remove_punc)
df['clean_abstract'] = df['abstract'].apply(remove_punc)

In [9]:
# remove digital
df['clean_content'] = df['clean_content'].apply(remove_digital)
df['clean_title'] = df['clean_title'].apply(remove_digital)
df['clean_abstract'] = df['clean_abstract'].apply(remove_digital)

In [10]:
# df['event_time'] =  pd.to_datetime(df['event_time'])
# df.sort_values(by='event_time', inplace=True)

In [11]:
# save clean data
df.to_csv(input_path+'clean_df.csv', index=False)

In [12]:
df[['key_label', 'title', 'abstract', 'content', 'industry_name_sc', 'concept_name_sc']].head(4)

,key_label,title,abstract,content,industry_name_sc,concept_name_sc
0,0,Market Movers | Solar stocks surge Thursday af...,Solar stocks were surging on Thursday morning ...,"<div class=""ftEditor""><p>11:02 AM EDT, 07/28/2...","[""太阳能"",""电气设备及零件""]","[""法国巴黎银行持仓"",""昨日强势股"",""光伏太阳能"",""氢能源"",""荷兰全球保险持仓"",""..."
1,0,COSCO Shipping Parent Lifts Stake in Firm to O...,"06:54 AM EDT, 07/22/2022 (MT Newswires) -- COS...","<p>06:54 AM EDT, 07/22/2022 (MT Newswires) -- ...","[""航运及港口"",""航运港口""]","[""一带一路"",""港口运输股"",""摘帽"",""转融券标的"",""国企改革"",""BDI指数"",""证..."
2,0,Does COSCO SHIPPING Holdings (HKG:1919) Have A...,The external fund manager backed by Berkshire ...,<p>The external fund manager backed by Berkshi...,"[""航运及港口""]","[""一带一路"",""港口运输股""]"
3,0,Does COSCO SHIPPING Holdings (HKG:1919) Have A...,The external fund manager backed by Berkshire ...,The external fund manager backed by Berkshire ...,"[""航运港口""]","[""一带一路"",""摘帽"",""转融券标的"",""国企改革"",""BDI指数"",""证金持股"",""上证..."


In [13]:
df[['key_label', 'clean_title', 'clean_abstract', 'clean_content', 'tags']].head(4)

,key_label,clean_title,clean_abstract,clean_content,tags
0,0,Market Movers Solar stocks surge Thursday af...,Solar stocks were surging on Thursday morning ...,div class ftEditor p AM EDT MT News...,太阳能 电气设备及零件 法国巴黎银行持仓 昨日强势股 光伏太阳能 氢能源 荷兰全球保险持仓 ...
1,0,COSCO Shipping Parent Lifts Stake in Firm to O...,AM EDT MT Newswires COSCO Shipping ...,p AM EDT MT Newswires COSCO Shippi...,航运及港口 航运港口 一带一路 港口运输股 摘帽 转融券标的 国企改革 BDI指数 证金持股...
2,0,Does COSCO SHIPPING Holdings HKG Have A Hea...,The external fund manager backed by Berkshire ...,p The external fund manager backed by Berkshi...,航运及港口 一带一路 港口运输股
3,0,Does COSCO SHIPPING Holdings HKG Have A Hea...,The external fund manager backed by Berkshire ...,The external fund manager backed by Berkshire ...,航运港口 一带一路 摘帽 转融券标的 国企改革 BDI指数 证金持股 上证50 AH股 进口...
